In [153]:
import os
import globP
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.data_processing_bronze_table
import utils.data_processing_silver_table
import utils.data_processing_gold_table


In [154]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

In [155]:
# create bronze datalake
bronze_lms_directory = "datamart/bronze/lms/"

if not os.path.exists(bronze_lms_directory):
    os.makedirs(bronze_lms_directory)

In [156]:
# ----------------------------
# 1️⃣ Read lms_loan_daily.csv
# ----------------------------
lms_path = "data/lms_loan_daily.csv"
df_lms = spark.read.csv(lms_path, header=True, inferSchema=True)
print("✅ LMS Loan Daily Data:")
df_lms.show(5, truncate=False)
df_lms.printSchema()

# ----------------------------
# 2️⃣ Read features_attributes.csv
# ----------------------------
attr_path = "data/features_attributes.csv"
df_attr = spark.read.csv(attr_path, header=True, inferSchema=True)
print("\n✅ Customer Attributes Data:")
df_attr.show(5, truncate=False)
df_attr.printSchema()

# ----------------------------
# 3️⃣ Read features_financials.csv
# ----------------------------
fin_path = "data/features_financials.csv"
df_fin = spark.read.csv(fin_path, header=True, inferSchema=True)
print("\n✅ Financial Features Data:")
df_fin.show(5, truncate=False)
df_fin.printSchema()

# ----------------------------
# 4️⃣ Read feature_clickstream.csv
# ----------------------------
click_path = "data/feature_clickstream.csv"
df_click = spark.read.csv(click_path, header=True, inferSchema=True)
print("\n✅ Clickstream Features Data:")
df_click.show(5, truncate=False)
df_click.printSchema()

# ----------------------------
# Optional: Count rows for a quick sanity check
# ----------------------------
print(f"""
Row Counts:
  LMS: {df_lms.count()}
  ATTR: {df_attr.count()}
  FIN: {df_fin.count()}
  CLICK: {df_click.count()}
""")



✅ LMS Loan Daily Data:
+---------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+-------------+
|loan_id              |Customer_ID|loan_start_date|tenure|installment_num|loan_amt|due_amt|paid_amt|overdue_amt|balance|snapshot_date|
+---------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+-------------+
|CUS_0x1000_2023_05_01|CUS_0x1000 |2023-05-01     |10    |0              |10000   |0.0    |0.0     |0.0        |10000.0|2023-05-01   |
|CUS_0x1000_2023_05_01|CUS_0x1000 |2023-05-01     |10    |1              |10000   |1000.0 |1000.0  |0.0        |9000.0 |2023-06-01   |
|CUS_0x1000_2023_05_01|CUS_0x1000 |2023-05-01     |10    |2              |10000   |1000.0 |1000.0  |0.0        |8000.0 |2023-07-01   |
|CUS_0x1000_2023_05_01|CUS_0x1000 |2023-05-01     |10    |3              |10000   |1000.0 |0.0     |1000.0     |8000.0 |2023-08-01   |
|CUS_0x1000_2023_05_01|CUS_0x100

In [157]:
df_lms.count()

137500

In [158]:
from pyspark.sql import functions as F

df_lms.select(
    F.min("snapshot_date").alias("min_snapshot_date"),
    F.max("snapshot_date").alias("max_snapshot_date"),
    F.min("loan_start_date").alias("min_loan_start_date"),
    F.max("loan_start_date").alias("max_loan_start_date")
).show()


+-----------------+-----------------+-------------------+-------------------+
|min_snapshot_date|max_snapshot_date|min_loan_start_date|max_loan_start_date|
+-----------------+-----------------+-------------------+-------------------+
|       2023-01-01|       2025-11-01|         2023-01-01|         2025-01-01|
+-----------------+-----------------+-------------------+-------------------+



In [159]:
df_lms_sorted_desc = df_lms.orderBy(F.col("snapshot_date").desc())
df_lms_sorted_desc.show(10, truncate=False)

+---------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+-------------+
|loan_id              |Customer_ID|loan_start_date|tenure|installment_num|loan_amt|due_amt|paid_amt|overdue_amt|balance|snapshot_date|
+---------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+-------------+
|CUS_0x11c1_2025_01_01|CUS_0x11c1 |2025-01-01     |10    |10             |10000   |1000.0 |1000.0  |0.0        |0.0    |2025-11-01   |
|CUS_0x8f35_2025_01_01|CUS_0x8f35 |2025-01-01     |10    |10             |10000   |1000.0 |1000.0  |0.0        |0.0    |2025-11-01   |
|CUS_0x526a_2025_01_01|CUS_0x526a |2025-01-01     |10    |10             |10000   |1000.0 |1000.0  |0.0        |0.0    |2025-11-01   |
|CUS_0x92c8_2025_01_01|CUS_0x92c8 |2025-01-01     |10    |10             |10000   |1000.0 |1000.0  |0.0        |0.0    |2025-11-01   |
|CUS_0x15c5_2025_01_01|CUS_0x15c5 |2025-01-01     |10  

In [160]:
from pyspark.sql import functions as F

df_lms.select("snapshot_date") \
    .distinct() \
    .orderBy("snapshot_date") \
    .show(1000, truncate=False)


+-------------+
|snapshot_date|
+-------------+
|2023-01-01   |
|2023-02-01   |
|2023-03-01   |
|2023-04-01   |
|2023-05-01   |
|2023-06-01   |
|2023-07-01   |
|2023-08-01   |
|2023-09-01   |
|2023-10-01   |
|2023-11-01   |
|2023-12-01   |
|2024-01-01   |
|2024-02-01   |
|2024-03-01   |
|2024-04-01   |
|2024-05-01   |
|2024-06-01   |
|2024-07-01   |
|2024-08-01   |
|2024-09-01   |
|2024-10-01   |
|2024-11-01   |
|2024-12-01   |
|2025-01-01   |
|2025-02-01   |
|2025-03-01   |
|2025-04-01   |
|2025-05-01   |
|2025-06-01   |
|2025-07-01   |
|2025-08-01   |
|2025-09-01   |
|2025-10-01   |
|2025-11-01   |
+-------------+



In [161]:
dates_str_lst = (
    df_lms.select("snapshot_date")
    .distinct()
    .orderBy("snapshot_date")
    .rdd.flatMap(lambda x: x)
    .collect()
)

dates_str_lst

[datetime.date(2023, 1, 1),
 datetime.date(2023, 2, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 4, 1),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 6, 1),
 datetime.date(2023, 7, 1),
 datetime.date(2023, 8, 1),
 datetime.date(2023, 9, 1),
 datetime.date(2023, 10, 1),
 datetime.date(2023, 11, 1),
 datetime.date(2023, 12, 1),
 datetime.date(2024, 1, 1),
 datetime.date(2024, 2, 1),
 datetime.date(2024, 3, 1),
 datetime.date(2024, 4, 1),
 datetime.date(2024, 5, 1),
 datetime.date(2024, 6, 1),
 datetime.date(2024, 7, 1),
 datetime.date(2024, 8, 1),
 datetime.date(2024, 9, 1),
 datetime.date(2024, 10, 1),
 datetime.date(2024, 11, 1),
 datetime.date(2024, 12, 1),
 datetime.date(2025, 1, 1),
 datetime.date(2025, 2, 1),
 datetime.date(2025, 3, 1),
 datetime.date(2025, 4, 1),
 datetime.date(2025, 5, 1),
 datetime.date(2025, 6, 1),
 datetime.date(2025, 7, 1),
 datetime.date(2025, 8, 1),
 datetime.date(2025, 9, 1),
 datetime.date(2025, 10, 1),
 datetime.date(2025, 11, 1)]

In [162]:
snapshot_dates = (
        df_lms
        .select(F.col("snapshot_date").cast("string"))
        .distinct()
        .orderBy("snapshot_date")
        .rdd.flatMap(lambda x: x)
        .collect()
    )
snapshot_dates 

['2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01',
 '2024-01-01',
 '2024-02-01',
 '2024-03-01',
 '2024-04-01',
 '2024-05-01',
 '2024-06-01',
 '2024-07-01',
 '2024-08-01',
 '2024-09-01',
 '2024-10-01',
 '2024-11-01',
 '2024-12-01',
 '2025-01-01',
 '2025-02-01',
 '2025-03-01',
 '2025-04-01',
 '2025-05-01',
 '2025-06-01',
 '2025-07-01',
 '2025-08-01',
 '2025-09-01',
 '2025-10-01',
 '2025-11-01']

In [163]:
snapshot_dates = (
        df_attr
        .select(F.col("snapshot_date").cast("string"))
        .distinct()
        .orderBy("snapshot_date")
        .rdd.flatMap(lambda x: x)
        .collect()
    )
snapshot_dates 

['2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01',
 '2024-01-01',
 '2024-02-01',
 '2024-03-01',
 '2024-04-01',
 '2024-05-01',
 '2024-06-01',
 '2024-07-01',
 '2024-08-01',
 '2024-09-01',
 '2024-10-01',
 '2024-11-01',
 '2024-12-01',
 '2025-01-01']

In [164]:
df_attr_202302 = df_attr.filter(F.col("snapshot_date") == "2023-02-01")
df_attr_202302.show(10, truncate=False)

+-----------+-----------------+---+-----------+------------+-------------+
|Customer_ID|Name             |Age|SSN        |Occupation  |snapshot_date|
+-----------+-----------------+---+-----------+------------+-------------+
|CUS_0x10aa |Carey Gillama    |20 |101-00-4424|Musician    |2023-02-01   |
|CUS_0x113e |Ransdell Piersono|45 |542-14-0249|Manager     |2023-02-01   |
|CUS_0x1140 |Jennifer Sabal   |31 |141-17-2233|Accountant  |2023-02-01   |
|CUS_0x1155 |Wohld            |41 |189-76-0870|Developer   |2023-02-01   |
|CUS_0x1252 |Irene Preisingera|28 |938-09-5300|Entrepreneur|2023-02-01   |
|CUS_0x129b |N.w              |24 |#F%$D@*&8  |Entrepreneur|2023-02-01   |
|CUS_0x12a6 |Natem            |25 |191-09-1571|Manager     |2023-02-01   |
|CUS_0x135c |Brownc           |41 |594-39-3524|_______     |2023-02-01   |
|CUS_0x1452 |Sarah Whited     |41 |976-05-6851|Architect   |2023-02-01   |
|CUS_0x14d5 |Kirstinz         |37 |476-48-0347|Engineer    |2023-02-01   |
+-----------+------------

In [165]:
df_attr.show()

+-----------+-----------------+---+-----------+-------------+-------------+
|Customer_ID|             Name|Age|        SSN|   Occupation|snapshot_date|
+-----------+-----------------+---+-----------+-------------+-------------+
| CUS_0x1000|   Alistair Barrf| 18|913-74-1218|       Lawyer|   2023-05-01|
| CUS_0x1009|           Arunah| 26|063-67-6938|     Mechanic|   2025-01-01|
| CUS_0x100b|         Shirboni| 19|  #F%$D@*&8|Media_Manager|   2024-03-01|
| CUS_0x1011|        Schneyerh| 44|793-05-8223|       Doctor|   2023-11-01|
| CUS_0x1013|         Cameront| 44|930-49-9615|     Mechanic|   2023-12-01|
| CUS_0x1015|          Holtono| 27|810-97-7024|   Journalist|   2023-08-01|
| CUS_0x1018|      Felsenthalq| 15|731-19-8119|   Accountant|   2023-11-01|
| CUS_0x1026|          Josephv| 52|500-62-9044|      Manager|   2023-10-01|
| CUS_0x102d| Neil Chatterjeex| 31|692-71-7552| Entrepreneur|   2024-01-01|
| CUS_0x102e|            Rhysn| 26|  #F%$D@*&8|    Scientist|   2024-04-01|
| CUS_0x1032

In [166]:
df_attr.filter(F.col("SSN") == "#F%$D@*&8").count()

703

In [167]:
df_attr = df_attr.withColumn(
    "SSN",
    F.when(F.col("SSN") == "#F%$D@*&8", "000-00-0000")
     .otherwise(F.col("SSN"))
)

In [168]:
df_attr.show()

+-----------+-----------------+---+-----------+-------------+-------------+
|Customer_ID|             Name|Age|        SSN|   Occupation|snapshot_date|
+-----------+-----------------+---+-----------+-------------+-------------+
| CUS_0x1000|   Alistair Barrf| 18|913-74-1218|       Lawyer|   2023-05-01|
| CUS_0x1009|           Arunah| 26|063-67-6938|     Mechanic|   2025-01-01|
| CUS_0x100b|         Shirboni| 19|000-00-0000|Media_Manager|   2024-03-01|
| CUS_0x1011|        Schneyerh| 44|793-05-8223|       Doctor|   2023-11-01|
| CUS_0x1013|         Cameront| 44|930-49-9615|     Mechanic|   2023-12-01|
| CUS_0x1015|          Holtono| 27|810-97-7024|   Journalist|   2023-08-01|
| CUS_0x1018|      Felsenthalq| 15|731-19-8119|   Accountant|   2023-11-01|
| CUS_0x1026|          Josephv| 52|500-62-9044|      Manager|   2023-10-01|
| CUS_0x102d| Neil Chatterjeex| 31|692-71-7552| Entrepreneur|   2024-01-01|
| CUS_0x102e|            Rhysn| 26|000-00-0000|    Scientist|   2024-04-01|
| CUS_0x1032

In [169]:
df_attr.select("Occupation") \
    .distinct() \
    .orderBy("Occupation") \
    .show(100, truncate=False)


+-------------+
|Occupation   |
+-------------+
|Accountant   |
|Architect    |
|Developer    |
|Doctor       |
|Engineer     |
|Entrepreneur |
|Journalist   |
|Lawyer       |
|Manager      |
|Mechanic     |
|Media_Manager|
|Musician     |
|Scientist    |
|Teacher      |
|Writer       |
|_______      |
+-------------+



In [170]:
df_attr.filter(F.col("Occupation") == "_______").count()


880

In [171]:
df_attr = df_attr.withColumn(
    "Occupation",
    F.when(F.col("Occupation") == "_______", "Unemployed")
     .otherwise(F.col("Occupation"))
)

In [172]:
df_attr.select("Occupation") \
    .distinct() \
    .orderBy("Occupation") \
    .show(100, truncate=False)


+-------------+
|Occupation   |
+-------------+
|Accountant   |
|Architect    |
|Developer    |
|Doctor       |
|Engineer     |
|Entrepreneur |
|Journalist   |
|Lawyer       |
|Manager      |
|Mechanic     |
|Media_Manager|
|Musician     |
|Scientist    |
|Teacher      |
|Unemployed   |
|Writer       |
+-------------+



In [173]:
df_attr.printSchema()


root
 |-- Customer_ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- snapshot_date: date (nullable = true)



In [174]:
column_type_map = {
    "Customer_ID": StringType(),
    "Name": StringType(),
    "Age": IntegerType(),
    "SSN": StringType(),
    "Occupation": StringType(),
    "snapshot_date": DateType(),
}

In [175]:
df_attr.filter(~F.col("Age").rlike("^[0-9]+$")).select("Customer_ID", "Name", "Age").show(50, truncate=False)


+-----------+----------------------+-----+
|Customer_ID|Name                  |Age  |
+-----------+----------------------+-----+
|CUS_0x1032 |Wahbap                |40_  |
|CUS_0x1057 |David Sheppardv       |46_  |
|CUS_0x10e7 |Carewj                |3843_|
|CUS_0x10ee |Hudsonb               |30_  |
|CUS_0x111c |Deepaa                |24_  |
|CUS_0x1135 |Baertleinc            |22_  |
|CUS_0x1139 |Copleyp               |32_  |
|CUS_0x11ac |Liana B.v             |26_  |
|CUS_0x1204 |Lashg                 |46_  |
|CUS_0x1288 |Taylorq               |34_  |
|CUS_0x13a8 |Baileyz               |41_  |
|CUS_0x13c2 |Menonv                |40_  |
|CUS_0x13e4 |Edward Krudyl         |1248_|
|CUS_0x1430 |Chiango               |30_  |
|CUS_0x14a3 |Prustyx               |-500 |
|CUS_0x14f4 |Richardc              |23_  |
|CUS_0x14f5 |Richard Leongd        |41_  |
|CUS_0x157d |Thompsonf             |24_  |
|CUS_0x1600 |Megan Daviesb         |30_  |
|CUS_0x1604 |Sam Forgioneb         |43_  |
|CUS_0x163c

In [176]:
df_attr.filter((F.col("Age") < 0) | (F.col("Age") > 100)) \
    .select("Age") \
    .distinct() \
    .orderBy("Age") \
    .show()

+----+
| Age|
+----+
|-500|
|1004|
|1022|
|1066|
|1087|
|1094|
|1149|
|1203|
|1220|
|1265|
|1388|
|1418|
|1520|
|1683|
| 169|
|1733|
|1792|
|1810|
|1814|
|1990|
+----+
only showing top 20 rows



In [177]:
df_attr = df_attr.withColumn(
    "Age",
    F.when((F.col("Age") >= 0) & (F.col("Age") <= 100), F.col("Age"))
     .otherwise(None)
)


In [178]:
df_attr.filter((F.col("Age") < 0) | (F.col("Age") > 100)) \
    .select("Age") \
    .distinct() \
    .orderBy("Age") \
    .show()

+---+
|Age|
+---+
+---+



In [179]:
column_type_map = {
    "Customer_ID": StringType(),
    "Name": StringType(),
    "Age": IntegerType(),
    "SSN": StringType(),
    "Occupation": StringType(),
    "snapshot_date": DateType(),
}

for column, new_type in column_type_map.items():
        df_attr = df_attr.withColumn(column, col(column).cast(new_type))

In [180]:
df_attr.dtypes

[('Customer_ID', 'string'),
 ('Name', 'string'),
 ('Age', 'int'),
 ('SSN', 'string'),
 ('Occupation', 'string'),
 ('snapshot_date', 'date')]

# DF FIN

In [242]:
def clean_numeric_column(df, column_name):
    """
    Cleans a numeric column in a PySpark DataFrame:
    1. Removes stray underscores from numeric values.
    2. Casts the column to FloatType.
    3. Uses IQR to detect outliers.
    4. Replaces outliers with None.
    
    Args:
        df (DataFrame): Input PySpark DataFrame
        column_name (str): Name of the numeric column to clean

    Returns:
        DataFrame: Cleaned DataFrame with outliers replaced by None
    """
    # 1️⃣ Remove underscores and cast to float
    dtype_lookup = dict(df.dtypes).get(column_name)
    if dtype_lookup == "string":
        df = df.withColumn(column_name, F.regexp_replace(F.col(column_name), "_", ""))

    df = df.withColumn(column_name, F.col(column_name).cast(FloatType()))

    # 2️⃣ Compute IQR boundaries
    q1, q3 = df.approxQuantile(column_name, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    # 3️⃣ Replace outliers with None
    df = df.withColumn(
        column_name,
        F.when((F.col(column_name) >= lower) & (F.col(column_name) <= upper),
               F.col(column_name))
         .otherwise(F.lit(None))
    )

    print(f"[CLEAN] Column '{column_name}' cleaned. IQR bounds: ({lower:.2f}, {upper:.2f})")

    return df

In [236]:
df_fin = spark.read.csv(fin_path, header=True, inferSchema=True)

In [243]:
df_fin.dtypes

[('Customer_ID', 'string'),
 ('Annual_Income', 'string'),
 ('Monthly_Inhand_Salary', 'double'),
 ('Num_Bank_Accounts', 'int'),
 ('Num_Credit_Card', 'int'),
 ('Interest_Rate', 'int'),
 ('Num_of_Loan', 'string'),
 ('Type_of_Loan', 'string'),
 ('Delay_from_due_date', 'int'),
 ('Num_of_Delayed_Payment', 'string'),
 ('Changed_Credit_Limit', 'string'),
 ('Num_Credit_Inquiries', 'double'),
 ('Credit_Mix', 'string'),
 ('Outstanding_Debt', 'string'),
 ('Credit_Utilization_Ratio', 'double'),
 ('Credit_History_Age', 'string'),
 ('Payment_of_Min_Amount', 'string'),
 ('Total_EMI_per_month', 'double'),
 ('Amount_invested_monthly', 'string'),
 ('Payment_Behaviour', 'string'),
 ('Monthly_Balance', 'string'),
 ('snapshot_date', 'date')]

In [244]:


df_fin = clean_numeric_column(df_fin, "Annual_Income")
df_fin = clean_numeric_column(df_fin, "Monthly_Inhand_Salary")
df_fin = clean_numeric_column(df_fin, "Num_Bank_Accounts")
df_fin = clean_numeric_column(df_fin, "Num_Credit_Card")
df_fin = clean_numeric_column(df_fin, "Interest_Rate")
df_fin = clean_numeric_column(df_fin, "Num_of_Loan")
df_fin = clean_numeric_column(df_fin, "Delay_from_due_date")
df_fin = clean_numeric_column(df_fin, "Num_of_Delayed_Payment")
df_fin = clean_numeric_column(df_fin, "Changed_Credit_Limit")
df_fin = clean_numeric_column(df_fin, "Num_Credit_Inquiries")
df_fin = clean_numeric_column(df_fin, "Outstanding_Debt")
df_fin = clean_numeric_column(df_fin, "Credit_Utilization_Ratio")
df_fin = clean_numeric_column(df_fin, "Total_EMI_per_month")
df_fin = clean_numeric_column(df_fin, "Amount_invested_monthly")
df_fin = clean_numeric_column(df_fin, "Monthly_Balance")



[CLEAN] Column 'Annual_Income' cleaned. IQR bounds: (-59626.93, 150522.66)
[CLEAN] Column 'Monthly_Inhand_Salary' cleaned. IQR bounds: (-4820.54, 12303.13)
[CLEAN] Column 'Num_Bank_Accounts' cleaned. IQR bounds: (-3.00, 13.00)
[CLEAN] Column 'Num_Credit_Card' cleaned. IQR bounds: (-0.50, 11.50)
[CLEAN] Column 'Interest_Rate' cleaned. IQR bounds: (-12.50, 39.50)
[CLEAN] Column 'Num_of_Loan' cleaned. IQR bounds: (-5.00, 11.00)
[CLEAN] Column 'Delay_from_due_date' cleaned. IQR bounds: (-17.00, 55.00)
[CLEAN] Column 'Num_of_Delayed_Payment' cleaned. IQR bounds: (-4.50, 31.50)
[CLEAN] Column 'Changed_Credit_Limit' cleaned. IQR bounds: (-8.52, 28.40)
[CLEAN] Column 'Num_Credit_Inquiries' cleaned. IQR bounds: (-7.50, 20.50)
[CLEAN] Column 'Outstanding_Debt' cleaned. IQR bounds: (-1474.93, 3928.24)
[CLEAN] Column 'Credit_Utilization_Ratio' cleaned. IQR bounds: (14.98, 49.45)
[CLEAN] Column 'Total_EMI_per_month' cleaned. IQR bounds: (-173.28, 369.76)
[CLEAN] Column 'Amount_invested_monthly' cle

In [245]:
df_fin = df_fin.withColumn(
    "Credit_History_Age",
    F.regexp_replace(F.col("Credit_History_Age"), "_", "")  # remove underscores if any
)

df_fin = df_fin.withColumn(
    "Years",
    F.regexp_extract(F.col("Credit_History_Age"), r"(\d+)\s+Years", 1).cast("int")
).withColumn(
    "Months",
    F.regexp_extract(F.col("Credit_History_Age"), r"(\d+)\s+Months", 1).cast("int")
)

df_fin = df_fin.withColumn(
    "Credit_History_Age_Months",
    (F.col("Years") * 12 + F.col("Months"))
)

# Drop the old columns if you want
df_fin = df_fin.drop("Years", "Months", "Credit_History_Age")

# (Optional) rename for simplicity
df_fin = df_fin.withColumnRenamed("Credit_History_Age_Months", "Credit_History_Age")

df_fin.select("Customer_ID", "Credit_History_Age").show(10, truncate=False)


+-----------+------------------+
|Customer_ID|Credit_History_Age|
+-----------+------------------+
|CUS_0x1000 |129               |
|CUS_0x1009 |372               |
|CUS_0x100b |190               |
|CUS_0x1011 |190               |
|CUS_0x1013 |214               |
|CUS_0x1015 |257               |
|CUS_0x1018 |171               |
|CUS_0x1026 |248               |
|CUS_0x102d |363               |
|CUS_0x102e |274               |
+-----------+------------------+
only showing top 10 rows



In [246]:
df_fin = df_fin.withColumn(
    "Payment_of_Min_Amount",
    F.trim(F.lower(F.col("Payment_of_Min_Amount")))  # normalize case
)

df_fin = df_fin.withColumn(
    "Payment_of_Min_Amount",
    F.when(F.col("Payment_of_Min_Amount").isin("yes", "y"), "Yes")
     .when(F.col("Payment_of_Min_Amount").isin("no", "n"), "No")
     .when(F.col("Payment_of_Min_Amount").isin("nm", "not mentioned", "na", "none"), None)
     .otherwise(F.col("Payment_of_Min_Amount"))  # keep valid ones
)

df_fin.select("Payment_of_Min_Amount").distinct().show()

+---------------------+
|Payment_of_Min_Amount|
+---------------------+
|                   No|
|                  Yes|
|                 NULL|
+---------------------+



In [248]:
df_fin = df_fin.withColumn(
    "Payment_Behaviour",
    F.when(F.col("Payment_Behaviour") == "!@9#%8", "Unknown")
     .otherwise(F.col("Payment_Behaviour"))
)


In [222]:
df_fin.filter(F.col("Num_Credit_Card") > 10).select().show()


++
||
++
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
++
only showing top 20 rows



In [223]:
df_fin = df_fin.withColumn(
    "Num_Bank_Accounts",
    F.when(F.col("Num_Bank_Accounts") > 10, 15)
     .otherwise(F.col("Num_Bank_Accounts"))
)

In [224]:
df_fin = df_fin.withColumn(
    "Num_Credit_Card",
    F.when(F.col("Num_Credit_Card") > 10, 10)
     .otherwise(F.col("Num_Credit_Card"))
)

In [225]:
df_fin.groupBy("Num_Credit_Card") \
      .count() \
      .orderBy("Num_Credit_Card") \
      .show(100)


+---------------+-----+
|Num_Credit_Card|count|
+---------------+-----+
|              0|    4|
|              1|  259|
|              2|  277|
|              3| 1647|
|              4| 1765|
|              5| 2302|
|              6| 2064|
|              7| 2072|
|              8|  611|
|              9|  586|
|             10|  913|
+---------------+-----+



In [226]:
q1, q3 = df_fin.approxQuantile("Interest_Rate", [0.25, 0.75], 0.01)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

df_outliers = df_fin.filter((F.col("Interest_Rate") < lower) | (F.col("Interest_Rate") > upper))
print(df_outliers.select("Interest_Rate").show(100))

df_inliers = df_fin.filter((F.col("Interest_Rate") > lower) & (F.col("Interest_Rate") < upper))
print(df_inliers.select("Interest_Rate").show(100))

+-------------+
|Interest_Rate|
+-------------+
|         1508|
|          498|
|         4041|
|         5240|
|          224|
|         1169|
|         5059|
|          422|
|         4985|
|         5773|
|          288|
|         3058|
|         5424|
|         3813|
|         2711|
|         4478|
|         2875|
|         4962|
|         4254|
|         5663|
|         2398|
|         2674|
|         1615|
|         5616|
|          207|
|         2865|
|         1292|
|         2577|
|         1922|
|         1663|
|         5774|
|         2299|
|          262|
|         4344|
|         3043|
|         2851|
|         1313|
|         1091|
|          926|
|          356|
|         2724|
|         3400|
|          954|
|         4586|
|          981|
|         1708|
|          890|
|         5024|
|         1106|
|         1172|
|         4195|
|         2660|
|          935|
|          570|
|         1292|
|         1884|
|          450|
|         5633|
|         2417|
|       

In [227]:
upper

39.5

In [228]:
df_fin.select("Interest_Rate").describe().show()

+-------+------------------+
|summary|     Interest_Rate|
+-------+------------------+
|  count|             12500|
|   mean|          73.21336|
| stddev|468.68222710052765|
|    min|                 1|
|    max|              5789|
+-------+------------------+



In [229]:
q1, q3 = df_fin.approxQuantile("Interest_Rate", [0.25, 0.75], 0.01)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

df_outliers = df_fin.filter((F.col("Interest_Rate") < lower) | (F.col("Interest_Rate") > upper))
print(df_outliers.select("Interest_Rate").show(100))

df_inliers = df_fin.filter((F.col("Interest_Rate") > lower) & (F.col("Interest_Rate") < upper))
print(df_inliers.select("Interest_Rate").show(100))

+-------------+
|Interest_Rate|
+-------------+
|         1508|
|          498|
|         4041|
|         5240|
|          224|
|         1169|
|         5059|
|          422|
|         4985|
|         5773|
|          288|
|         3058|
|         5424|
|         3813|
|         2711|
|         4478|
|         2875|
|         4962|
|         4254|
|         5663|
|         2398|
|         2674|
|         1615|
|         5616|
|          207|
|         2865|
|         1292|
|         2577|
|         1922|
|         1663|
|         5774|
|         2299|
|          262|
|         4344|
|         3043|
|         2851|
|         1313|
|         1091|
|          926|
|          356|
|         2724|
|         3400|
|          954|
|         4586|
|          981|
|         1708|
|          890|
|         5024|
|         1106|
|         1172|
|         4195|
|         2660|
|          935|
|          570|
|         1292|
|         1884|
|          450|
|         5633|
|         2417|
|       

In [230]:
def clean_numeric_column(df, column_name):
    """
    Cleans a numeric column in a PySpark DataFrame:
    1. Removes stray underscores from numeric values.
    2. Casts the column to FloatType.
    3. Uses IQR to detect outliers.
    4. Replaces outliers with None.
    
    Args:
        df (DataFrame): Input PySpark DataFrame
        column_name (str): Name of the numeric column to clean

    Returns:
        DataFrame: Cleaned DataFrame with outliers replaced by None
    """
    # 1️⃣ Remove underscores and cast to float
    df = df.withColumn(column_name, F.regexp_replace(F.col(column_name), "_", ""))
    df = df.withColumn(column_name, F.col(column_name).cast(FloatType()))

    # 2️⃣ Compute IQR boundaries
    q1, q3 = df.approxQuantile(column_name, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    # 3️⃣ Replace outliers with None
    df = df.withColumn(
        column_name,
        F.when((F.col(column_name) >= lower) & (F.col(column_name) <= upper),
               F.col(column_name))
         .otherwise(F.lit(None))
    )

    print(f"[CLEAN] Column '{column_name}' cleaned. IQR bounds: ({lower:.2f}, {upper:.2f})")

    return df

In [231]:
df_fin = clean_numeric_column(df_fin, "Interest_Rate")

[CLEAN] Column 'Interest_Rate' cleaned. IQR bounds: (-12.50, 39.50)


In [232]:
df_fin = clean_numeric_column(df_fin, "Num_of_Loan")

[CLEAN] Column 'Num_of_Loan' cleaned. IQR bounds: (-5.00, 11.00)


In [233]:
df_fin.select("Num_of_Loan") \
      .distinct() \
      .orderBy("Num_of_Loan") \
      .show(100)

+-----------+
|Num_of_Loan|
+-----------+
|       NULL|
|        0.0|
|        1.0|
|        2.0|
|        3.0|
|        4.0|
|        5.0|
|        6.0|
|        7.0|
|        8.0|
|        9.0|
+-----------+



In [234]:
numeric_cols = [field.name for field in df_fin.schema.fields 
                if field.dataType.typeName() in ["integer", "double", "float", "long"]]

df_fin.select(numeric_cols).describe().show()


+-------+---------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+------------------------+-------------------+
|summary|Monthly_Inhand_Salary| Num_Bank_Accounts|   Num_Credit_Card|     Interest_Rate|       Num_of_Loan|Delay_from_due_date|Num_Credit_Inquiries|Credit_Utilization_Ratio|Total_EMI_per_month|
+-------+---------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+------------------------+-------------------+
|  count|                12500|             12500|             12500|             12230|             11933|              12500|               12500|                   12500|              12500|
|   mean|   4188.5923027131585|           5.49368|           5.64032|14.556255110384301|3.5290371239420097|           21.06088|            26.69528|       32.34926457800977| 1488.3942909941488|
| stddev|   3180.1476109204173